In [22]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [23]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)


llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_co

In [24]:
import psycopg2

# db_name = "vector_db"
# host = "localhost"
# password = "postgres"
# port = "5432"
# user = "postgres"
# # conn = psycopg2.connect(connection_string)
# conn = psycopg2.connect(
#     dbname=postgres,
#     host=host,
#     password=password,
#     port=port,
#     user=user,
# )
# conn.autocommit = True

# with conn.cursor() as c:
#     c.execute(f"DROP DATABASE IF EXISTS {db_name}")
#     c.execute(f"CREATE DATABASE {db_name}")


In [25]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

db_name = "vectordb"
host = "db"
password = "testpwd"
port = "5432"
user = "testuser"
vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

In [26]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

In [27]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))



In [28]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [29]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding


In [30]:
vector_store.add(nodes)

['b60b3af9-1f4a-4999-868d-ef9c1d3b0911',
 'd884f901-16da-438b-8b3c-d233c6b84f72',
 'a6c78910-7e0c-4427-bce0-7ef8da1d2176',
 '0cc02445-c028-4b6e-872e-6e2f09a98d17',
 '40031660-e878-4b2a-911d-2106583f8c3e',
 '58d51d7c-e77e-4bfd-916d-c784d643ab8b',
 'a68be30d-71c7-4651-9b67-d8933d9f7680',
 '59f350f3-0d9b-44f0-8f3a-272bb243533b',
 '6f518b81-80f2-4079-b458-ad0164ae3871',
 'f56376b9-2a2d-4e76-b614-97dc51986c94',
 '89b93795-a70d-4328-8b80-92dc3d38bdf5',
 '403e22f2-134a-4382-b2bc-0206c13df4a3',
 '64f63893-95b3-480a-86f3-3936fa3cde36',
 'b14c16a9-fafe-46ae-aa33-78d52046928e',
 'e6c78f2d-74bf-4835-97fa-7229be7e22e5',
 '3eaf33c2-7cc5-4bf0-9b17-af9aacce140d',
 '82c825ee-7c49-460a-a8b9-7e2a5a4ca640',
 '82b26bd6-99d7-4228-8c2c-d798e0f1c962',
 '4dcc615e-d05f-4617-83c8-7e7f981e1e4f',
 'c6f77b69-6420-444d-bf4e-b37f0b1d678f',
 '3fdff109-b8a5-4fc8-9da5-a55381ae4da3',
 '40702c00-3808-4388-b7a2-2df3e146129d',
 '18bebb53-20e6-45ae-abf0-82a966f1cb68',
 '29e79258-685b-4eab-b9ca-c5883f0800bb',
 '454d01a4-0bdb-

In [31]:
query_str = "Can you tell me about the key concepts for safety finetuning"


In [32]:
query_embedding = embed_model.get_query_embedding(query_str)


In [33]:
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [34]:
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())


TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary

In [35]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))


In [36]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [37]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)


In [38]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [39]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)



llama_print_timings:        load time =  104646.44 ms
llama_print_timings:      sample time =      39.48 ms /    74 runs   (    0.53 ms per token,  1874.18 tokens per second)
llama_print_timings: prompt eval time =  155699.80 ms /   763 tokens (  204.06 ms per token,     4.90 tokens per second)
llama_print_timings:        eval time =   30926.78 ms /    73 runs   (  423.65 ms per token,     2.36 tokens per second)
llama_print_timings:       total time =  186881.99 ms /   836 tokens


In [40]:
print(str(response))


 Based on the context information provided, Llama 2 performs on par or better than other open-source models such as PaLM (540B) on almost all benchmarks, but there is still a significant gap in performance between Llama 2 and closed-source models such as GPT-3.5 and GPT-4.


In [41]:
print(response.source_nodes[0].get_content())


Additionally, Llama 2 70B model outperforms all open-source models.
In addition to open-source models, we also compare Llama 2 70B results to closed-source models. As shown
in Table 4, Llama 2 70B is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant
gap on coding benchmarks. Llama 2 70B results are on par or better than PaLM (540B) (Chowdhery et al.,
2022) on almost all benchmarks. There is still a large gap in performance between Llama 2 70B and GPT-4
and PaLM-2-L.
We also analysed the potential data contamination and share the details in Section A.6.
Benchmark (shots)
GPT-3.5
GPT-4
PaLM
PaLM-2-L
Llama 2
MMLU (5-shot)
70.0
86.4
69.3
78.3
68.9
TriviaQA (1-shot)
–
–
81.4
86.1
85.0
Natural Questions (1-shot)
–
–
29.3
37.5
33.0
GSM8K (8-shot)
57.1
92.0
56.5
80.7
56.8
HumanEval (0-shot)
48.1
67.0
26.2
–
29.9
BIG-Bench Hard (3-shot)
–
–
52.3
65.7
51.2
Table 4: Comparison to closed-source models on academic benchmarks. Results for GPT-3.5 and GPT-4
are from OpenAI